<a href="https://colab.research.google.com/github/DajanaSanchez/BirdMigrationClimateAnalysis/blob/main/bird_land_climate_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import gc

# ✅ Load Bird + Land Data
df_merged = pd.read_parquet("/content/drive/MyDrive/bird_land_habitat_updated.parquet")

df_merged.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,geometry,index_right,Year_right,Date_right,Latitude_right,Longitude_right,Region,Land_Cover_Type,Land_Cover_Description,distance,Year_Diff,Has_Habitat_Change
6031513,32.793335,-97.492590,1998-12-01,1998,b'\x01\x01\x00\x00\x00\x8b\x155\x98\x86_X\xc0\...,2407646,2001,2001-01,32.776141,-97.489575,US,10,Grasslands,2301.304418,3,False
1941428,49.498783,-115.793117,2019-08-01,2019,b'\x01\x01\x00\x00\x00T\x1a\x8cl\xc2\xf2\\\xc0...,57030111,2019,2019-01,49.500736,-115.804837,Canada,8,Woody Savannas,1347.001098,0,False
6792659,46.161820,-118.955240,2018-01-01,2018,"b'\x01\x01\x00\x00\x00\xc5\xb4\xca\xa7""\xbd]\x...",53930540,2018,2018-01,46.145803,-118.957465,US,12,Croplands,2585.918796,0,False
5713478,41.588694,-80.406360,2019-03-01,2019,b'\x01\x01\x00\x00\x000\xda\x88\xcc\x01\x1aT\x...,57409860,2019,2019-01,41.589102,-80.425354,US,8,Woody Savannas,2115.347150,0,False
785999,33.574042,-111.537758,2022-02-01,2022,b'\x01\x01\x00\x00\x00p\xebn\x9ej\xe2[\xc0\xfe...,64313560,2022,2022-01,33.577320,-111.551293,US,7,Open Shrublands,1569.105188,0,True
6321896,38.550381,-77.295449,2016-04-01,2016,b'\x01\x01\x00\x00\x00\x1ei\x15\xa2\xe8RS\xc0(...,47775539,2016,2016-01,38.534610,-77.272727,US,13,Urban and Built-up,3381.691859,0,True
6352904,42.870838,-72.562380,2023-03-01,2023,b'\x01\x01\x00\x00\x00`\x9bC\x08\xfe#R\xc0F\x1...,70388607,2023,2023-01,42.891016,-72.568807,US,4,Deciduous Broadleaf Forest,3147.848682,0,False
4046248,38.963910,-74.961590,2021-10-01,2021,b'\x01\x01\x00\x00\x00I\x8a\xc8\xb0\x8a\xbdR\x...,67315440,2021,2021-01,38.985272,-74.970809,US,0,Water,3226.455208,0,False
4797167,40.740104,-84.044552,2015-06-01,2015,b'\x01\x01\x00\x00\x00\x95\xfe\xca\xef\xd9\x02...,44409607,2015,2015-01,40.737850,-84.028357,US,12,Croplands,1832.964174,0,False
6081357,37.374595,-77.362896,2023-02-01,2023,b'\x01\x01\x00\x00\x000ww\xae9WS\xc0\xf4\xdes\...,70652401,2023,2023-01,37.382916,-77.372811,US,13,Urban and Built-up,1605.310210,0,False


In [3]:
#dropping unneccessary columns
df_birdland_final = df_merged.drop(columns=['geometry', 'index_right', 'Year_right', 'Date_right', 'Latitude_right', 'Longitude_right', 'distance', 'Year_Diff'])

In [4]:
df_birdland_final.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,Region,Land_Cover_Type,Land_Cover_Description,Has_Habitat_Change
6514229,46.707971,-123.987783,2023-05-01,2023,US,2,Evergreen Broadleaf Forest,True
4994686,43.637568,-73.431367,2020-11-01,2020,US,4,Deciduous Broadleaf Forest,False
5634323,40.565945,-75.986072,2015-09-01,2015,US,4,Deciduous Broadleaf Forest,False
3546240,38.845460,-90.147774,2021-07-01,2021,US,13,Urban and Built-up,False
1099030,39.549225,-105.075753,2015-02-01,2015,US,13,Urban and Built-up,False
294120,44.656184,-66.773561,2023-08-01,2023,US,5,Mixed Forests,False
2084857,31.651697,-81.393436,2018-04-01,2018,US,8,Woody Savannas,False
3685597,45.573228,-94.554650,2017-07-01,2017,US,12,Croplands,False
2343530,39.703251,-95.729004,2024-01-01,2024,US,12,Croplands,False
4368740,42.300393,-75.467365,2004-10-01,2004,US,8,Woody Savannas,False


In [5]:
print(df_birdland_final.count())

Latitude_left             7498836
Longitude_left            7498836
Date_left                 7498836
Year_left                 7498836
Region                    7498836
Land_Cover_Type           7498836
Land_Cover_Description    7498836
Has_Habitat_Change        7498836
dtype: int64


In [6]:
# ✅ Convert to GeoDataFrame
bird_land_gdf = gpd.GeoDataFrame(
    df_birdland_final,
    geometry=gpd.points_from_xy(df_merged.Longitude_left, df_merged.Latitude_left),
    crs="EPSG:4326"
)

# ✅ Drop unneeded columns (geometry in binary format)
bird_land_gdf = bird_land_gdf.drop(columns=["geometry"], errors="ignore")

In [7]:
bird_land_gdf.count()

,0
Latitude_left,7498836
Longitude_left,7498836
Date_left,7498836
Year_left,7498836
Region,7498836
Land_Cover_Type,7498836
Land_Cover_Description,7498836
Has_Habitat_Change,7498836


In [8]:
# ✅ Load Climate Data from Parquet
climate_csv = "/content/drive/MyDrive/climate_merged_final.csv"

# Convert 'climate_merged_final_.csv' to Parquet
climate_parquet = pd.read_csv(climate_csv)

# Save as Parquet
climate_parquet.to_parquet("climate_merged_final.parquet", engine="pyarrow", index=False)

#drop unnecessary fields
final_climate_parquet = climate_parquet.drop(columns=['Station_ID', 'Elevation', 'State', 'StationName', 'TempFlags', 'Precip_MeasurementFlag', 'Precip_QCFlag', 'Precip_SourceFlag', 'Precip_SourceIndex', 'Precip_FirstYear', 'Precip_LastYear'])

final_climate_parquet.head()

<ipython-input-8-411e258a663f>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  climate_parquet = pd.read_csv(climate_csv)


,Date,Latitude,Longitude,Temp_celsius,Precip_mm
0,1961-01,48.9333,-123.75,5.01,372.3
1,1962-01,48.9333,-123.75,3.53,107.4
2,1963-01,48.9333,-123.75,1.55,33.5
3,1964-01,48.9333,-123.75,4.56,302.5
4,1965-01,48.9333,-123.75,2.73,169.4


In [9]:
# ✅ Round Latitude & Longitude to 6 decimal places
final_climate_parquet["Latitude"] = final_climate_parquet["Latitude"].round(6)
final_climate_parquet["Longitude"] = final_climate_parquet["Longitude"].round(6)

In [15]:
# ✅ Convert to GeoDataFrame
climate_gdf = final_climate_parquet[["Latitude", "Longitude", "Temp_celsius", "Precip_mm", "Date"]].copy()


# ✅ Convert Date to datetime format and extract Year
climate_gdf["Date"] = pd.to_datetime(climate_gdf["Date"], errors="coerce")
climate_gdf["Year_climate"] = climate_gdf["Date"].dt.year

# ✅ Convert to GeoDataFrame
climate_gdf = gpd.GeoDataFrame(
    climate_gdf,
    geometry=gpd.points_from_xy(climate_gdf.Longitude, climate_gdf.Latitude),
    crs="EPSG:4326"
)

# ✅ Convert numeric fields to float32 for memory efficiency
climate_gdf["Temp_celsius"] = climate_gdf["Temp_celsius"].astype("float32")
climate_gdf["Precip_mm"] = climate_gdf["Precip_mm"].astype("float32")

# ✅ Drop unnecessary columns (ensures better performance)
climate_gdf = climate_gdf.drop(columns=["geometry"], errors="ignore")

print(f"✅ Loaded Climate Data with {len(climate_gdf)} records.")

✅ Loaded Climate Data with 1484396 records.


In [16]:
climate_gdf.head()

,Latitude,Longitude,Temp_celsius,Precip_mm,Date,Year_climate
0,48.9333,-123.75,5.01,372.299988,1961-01-01,1961
1,48.9333,-123.75,3.53,107.400002,1962-01-01,1962
2,48.9333,-123.75,1.55,33.500000,1963-01-01,1963
3,48.9333,-123.75,4.56,302.500000,1964-01-01,1964
4,48.9333,-123.75,2.73,169.399994,1965-01-01,1965


In [66]:
import os
import pandas as pd
import numpy as np
import gc
from scipy.spatial import cKDTree

# ✅ Define Paths
bird_land_file = "/content/drive/MyDrive/bird_land_habitat_updated.parquet"
climate_file = "/content/drive/MyDrive/climate_merged_final.parquet"
output_dir = "/content/drive/MyDrive/bird_land_climate_chunks"
os.makedirs(output_dir, exist_ok=True)  # ✅ Ensure the output directory exists

chunk_size = 250000  # ✅ Process Bird + Land data in chunks

print("✅ Loading full Bird + Land dataset...")

# ✅ Load Bird + Land Data (ENSURE ALL NECESSARY FIELDS ARE INCLUDED)
bird_land_df = pd.read_parquet(
    bird_land_file,
    columns=["Latitude_left", "Longitude_left", "Date_left", "Year_left", "Region",
             "Land_Cover_Type", "Land_Cover_Description", "Has_Habitat_Change"]
)

# ✅ Load Climate Data
climate_df = pd.read_parquet(climate_file)

# ✅ Convert Climate Date & Extract Year
climate_df["Date"] = pd.to_datetime(climate_df["Date"], errors="coerce")
climate_df["Year_climate"] = climate_df["Date"].dt.year
climate_df["Date_ordinal"] = climate_df["Date"].map(lambda d: d.toordinal())  # Convert Date to numerical format

print(f"✅ Climate data loaded with {len(climate_df)} records.")

# ✅ Build KDTree for Climate Data (Based on Latitude, Longitude, and Date)
climate_tree = cKDTree(climate_df[["Longitude", "Latitude", "Date_ordinal"]].values)

print("✅ KDTree built for Climate Data.")

# ✅ Define function to find the nearest climate match using `k=1`
def get_nearest_climate(row):
    """
    Finds the nearest climate record for a given bird/land observation using k=1.
    Ensures the closest match in both space and time.
    """
    # Query KDTree for **only the closest** climate record
    _, idx = climate_tree.query([row["Longitude_left"], row["Latitude_left"], row["Date_ordinal"]], k=1)

    # Retrieve the closest match
    best_match = climate_df.iloc[idx].copy()

    # Compute year difference
    best_match["Year_Diff_Climate"] = abs(best_match["Year_climate"] - row["Year_left"])

    return pd.Series({
        "Latitude_climate": best_match["Latitude"],
        "Longitude_climate": best_match["Longitude"],
        "Date_climate": best_match["Date"],
        "Year_climate": best_match["Year_climate"],
        "Temp_celsius": best_match["Temp_celsius"],
        "Precip_mm": best_match["Precip_mm"],
        "Year_Diff_Climate": best_match["Year_Diff_Climate"]
    })

print("✅ Starting chunked processing...")

# ✅ Process Bird + Land in chunks manually
for i in range(0, len(bird_land_df), chunk_size):

    chunk_file = f"bird_land_climate_chunk_{i}.parquet"
    chunk_path = os.path.join(output_dir, chunk_file)

    print(f"🔹 Processing rows {i} to {i + chunk_size}...")

    # ✅ Select chunk of Bird + Land data
    chunk = bird_land_df.iloc[i: i + chunk_size].copy()

    # ✅ Convert Date & Extract Year
    chunk["Date_left"] = pd.to_datetime(chunk["Date_left"], errors="coerce")
    chunk["Year_left"] = chunk["Date_left"].dt.year
    chunk["Date_ordinal"] = chunk["Date_left"].map(lambda d: d.toordinal())  # Convert Date to numerical format

    # ✅ Apply KDTree nearest neighbor search (Finds closest match)
    climate_info = chunk.apply(get_nearest_climate, axis=1)

    # ✅ Merge climate data back to Bird + Land data
    bird_land_climate_chunk = pd.concat([chunk, climate_info], axis=1)

    print(f"✅ Spatial join complete. {len(bird_land_climate_chunk)} records after KDTree join.")

    # ✅ Save Each Chunk to Google Drive
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    bird_land_climate_chunk.to_parquet(chunk_path, index=False)
    print(f"✅ Processed and saved {len(bird_land_climate_chunk)} rows to {chunk_path}.")

    # ✅ Free Memory
    del chunk, climate_info, bird_land_climate_chunk
    gc.collect()

print("✅ Processing complete for all chunks.")

✅ Loading full Bird + Land dataset...
✅ Climate data loaded with 1484396 records.
✅ KDTree built for Climate Data.
✅ Starting chunked processing...
🔹 Processing rows 0 to 250000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_0.parquet.
🔹 Processing rows 250000 to 500000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_250000.parquet.
🔹 Processing rows 500000 to 750000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_500000.parquet.
🔹 Processing rows 750000 to 1000000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_cli

In [67]:
import os
import pandas as pd
import gc

# ✅ Define Paths
chunk_dir = "/content/drive/MyDrive/bird_land_climate_chunks"
final_output_file = "/content/drive/MyDrive/bird_land_climate_final.parquet"

print("✅ Checking for chunk files...")

# ✅ Get list of all chunked files
all_chunks = sorted([os.path.join(chunk_dir, f) for f in os.listdir(chunk_dir) if f.endswith(".parquet")])

print(f"✅ Found {len(all_chunks)} chunk files.")

if len(all_chunks) == 0:
    print("❌ No chunk files found. Ensure chunk processing is complete before merging.")
else:
    print("✅ Starting chunk merge...")

    # ✅ Initialize an empty list to store DataFrames
    merged_dfs = []

    # ✅ Read each chunk, merge them in memory-efficient way
    for chunk_file in all_chunks:
        print(f"🔹 Merging {chunk_file}...")

        # ✅ Read chunk
        df_chunk = pd.read_parquet(chunk_file)

        # ✅ Append to list
        merged_dfs.append(df_chunk)

        # ✅ Free Memory
        del df_chunk
        gc.collect()

    # ✅ Concatenate all chunks into final DataFrame
    final_df = pd.concat(merged_dfs, ignore_index=True)

    # ✅ Save the final dataset to Google Drive
    final_df.to_parquet(final_output_file, index=False)

    print(f"✅ Final dataset saved at: {final_output_file}")

    # ✅ Free Memory
    del final_df, merged_dfs
    gc.collect()

print("✅ Chunk merging complete!")

✅ Checking for chunk files...
✅ Found 30 chunk files.
✅ Starting chunk merge...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_0.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1000000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1500000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1750000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2000000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2500000.parquet...
🔹 Merging /content

In [68]:
df = pd.read_parquet(final_output_file)

In [76]:
df.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,Region,Land_Cover_Type,Land_Cover_Description,Has_Habitat_Change,Date_ordinal,Latitude_climate,Longitude_climate,Date_climate,Year_climate,Temp_celsius,Precip_mm,Year_Diff_Climate
5884312,38.013045,-78.970982,2020-09-01,2020,US,12,Croplands,False,737669,37.9975,-78.4656,2010-12-01,2010,-0.79,48.7,10
4569124,48.451706,-89.279291,2024-04-01,2024,US,8,Woody Savannas,False,738977,48.3667,-89.3333,2010-12-01,2010,-10.30,27.5,14
5413334,40.883135,-77.908988,2020-05-01,2020,US,4,Deciduous Broadleaf Forest,False,737546,41.0464,-78.4111,2010-12-01,2010,-4.20,45.7,10
3680720,40.856117,-96.875360,2022-03-01,2022,US,12,Croplands,False,738215,40.6953,-96.8542,2010-12-01,2010,-3.46,7.1,12
6921563,44.663877,-90.242086,2019-10-01,2019,US,14,Cropland/Natural Vegetation Mosaic,False,737333,44.6378,-90.1875,2010-12-01,2010,-8.21,9.8,9
6105789,43.085680,-72.439230,2016-08-01,2016,US,5,Mixed Forests,False,736177,43.3422,-72.5214,2010-12-01,2010,-3.92,109.8,6
5923172,38.769500,-77.049400,2003-10-01,2003,US,13,Urban and Built-up,False,731489,38.5036,-77.3050,2003-10-01,2003,13.65,69.6,0
3044222,38.816106,-90.133258,2017-10-01,2017,US,13,Urban and Built-up,False,736603,38.5639,-90.1486,2010-12-01,2010,-1.29,31.3,7
5334132,40.271668,-76.247735,2018-11-01,2018,US,12,Croplands,False,736999,40.1206,-76.2944,2010-12-01,2010,-0.59,50.8,8
519223,26.876909,-82.311426,2015-03-01,2015,US,0,Water,False,735658,26.9183,-81.9939,2010-12-01,2010,12.27,28.2,5


In [80]:
df.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,Region,Land_Cover_Description,Has_Habitat_Change,Temp_celsius,Precip_mm
7264948,32.622101,-85.472503,2024-09-01,2024,US,Woody Savannas,False,6.57,27.6
3176080,45.384546,-93.915087,2019-03-01,2019,US,Croplands,False,-10.17,23.7
4655891,58.301571,-134.421768,1991-08-01,1991,Canada,Evergreen Needleleaf Forest,False,11.18,55.4
6828361,43.186064,-89.901005,2013-04-01,2013,US,Woody Savannas,True,-7.69,23.0
1555506,38.985500,-87.170334,2021-05-01,2021,US,Deciduous Broadleaf Forest,False,-3.82,31.5
1536195,38.706662,-87.473818,2024-03-01,2024,US,Croplands,False,-2.13,33.3
4584504,49.065937,-122.967825,2024-11-01,2024,Canada,Water,True,5.71,153.8
1435438,42.462937,-92.925455,2024-05-01,2024,US,Croplands,False,-7.55,8.0
4083061,46.301525,-72.534828,2019-12-01,2019,US,Urban and Built-up,False,-5.69,115.4
1763998,42.610376,-81.798973,2023-03-01,2023,US,Croplands,False,-4.49,45.0


In [81]:
#rename columns
df = df.rename(columns={
    'Latitude_left': 'Latitude',
    'Longitude_left': 'Longitude',
    'Date_left': 'Observation_Date',
    'Year_left': 'Year'})

In [82]:
df.sample(10)

,Latitude,Longitude,Observation_Date,Year,Region,Land_Cover_Description,Has_Habitat_Change,Temp_celsius,Precip_mm
7227474,51.006307,-114.018289,2022-04-01,2022,Canada,Urban and Built-up,False,-8.42,2.6
3557893,39.109690,-74.708253,2010-10-01,2010,US,Woody Savannas,True,14.69,102.9
1953442,49.843455,-93.899168,2023-09-01,2023,Canada,Woody Savannas,False,-15.14,9.5
3056510,38.868589,-90.969234,2020-03-01,2020,US,Savannas,False,-2.23,22.3
232254,53.230752,-132.008607,2014-01-01,2014,Canada,Water,False,3.83,142.5
5176753,45.506488,-123.085186,2016-03-01,2016,US,Croplands,False,6.03,207.2
5368613,40.423956,-76.996028,2018-03-01,2018,US,Deciduous Broadleaf Forest,False,0.20,42.9
7133893,48.934050,-125.539948,2022-07-01,2022,US,Evergreen Needleleaf Forest,False,5.59,608.7
198982,50.218436,-63.889360,2006-07-01,2006,Canada,Water,False,15.85,92.5
562971,29.574222,-82.334182,2021-02-01,2021,US,Urban and Built-up,False,7.71,30.6


In [83]:
df.describe()

,Latitude,Longitude,Observation_Date,Year,Temp_celsius,Precip_mm
count,7.498836e+06,7.498836e+06,7498836,7.498836e+06,7.498836e+06,7.498836e+06
mean,4.238773e+01,-9.358986e+01,2019-06-21 17:59:32.958363136,2.019055e+03,-1.855947e-01,9.082817e+01
min,1.934370e+01,-1.790633e+02,1974-01-01 00:00:00,1.974000e+03,-9.999000e+01,0.000000e+00
25%,3.912949e+01,-1.140327e+02,2017-05-01 00:00:00,2.017000e+03,-4.310000e+00,3.070000e+01
50%,4.240090e+01,-8.596323e+01,2020-11-01 00:00:00,2.020000e+03,-9.200000e-01,5.680000e+01
75%,4.621175e+01,-7.676376e+01,2023-01-01 00:00:00,2.023000e+03,4.490000e+00,1.138000e+02
max,7.315889e+01,1.792826e+02,2024-12-01 00:00:00,2.024000e+03,3.572000e+01,1.323200e+03
std,6.207601e+00,2.055219e+01,NaN,5.087835e+00,6.481743e+00,9.878443e+01


In [84]:
#print final data frame to a parquet file in my Google Drive
df.to_parquet("/content/drive/MyDrive/final_merged_dataset.parquet")